In [6]:
import pandas as pd
import random
import scipy as sp
import numpy as np
import cvxpy as cvx
import mosek
import matplotlib.pyplot as plt 
import math
import warnings
import parameters
import simulations

#### Load Data

In [7]:
dfTripData = pd.read_csv("Data/location_data.csv")
dfLoadData = pd.read_csv("Data/use_data_2wk.csv") #2 weeks days, 1 min increment, 142 houses 
dfGenData = pd.read_csv("Data/annual_capfac_15MinInterpolation.csv") #representative solar day, 15 min interval
#DAM
dfAsRDdamPrices=pd.read_csv("Data/AS_Prices/DAM/20190701_20190801_PRC_AS_DAM_REGDOWN.csv")
dfAsRUdamPrices=pd.read_csv("Data/AS_Prices/DAM/20190701_20190801_PRC_AS_DAM_REGUP.csv")

#RDU/RU RTM
dfAsRDrtmPrices=pd.read_csv("Data/AS_Prices/RTM/20190701_20190731_PRC_INTVL_AS_RTM_REGDOWN.csv")
dfAsRUrtmPrices=pd.read_csv("Data/AS_Prices/RTM/20190701_20190731_PRC_INTVL_AS_RTM_REGUP.csv")  
dfCostElectric = pd.read_csv("Data/TOU_Rate.csv") #15 min interal 4-9 pm, I think this is tou B

#### Script Parameters

In [49]:
intNumHouseNodes = 5
intNumCommNodes = 2
intTotalNodes = intNumHouseNodes + intNumCommNodes
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 7 # kW
fltWorkRate = 7 # kW

# time and conversion intervals
int_minutes=int(15)
int_run_days=1 # day
int_run_hours=24 # hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  # convert kw data to kwh


# cost to charge
lsCostElectric = []
lsHolder = dfCostElectric.values.tolist()
for ind in range(int_run_days):
    for n in lsHolder:
        lsCostElectric.append(n[0])


In [51]:
# call functions to get Nodes Dataframe
#dfNodesTrips, intVeHouses = parameters.createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
#                    intProfiles, dfTripData)

#create df of loads for commercial & residential nodes
arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, \
lsNodes,dctHomeNode = parameters.convertLoadData(dfLoadData,dfNodesTrips, \
                                    int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes)

#create generation for 15 min increments for each node
arrHouseGen,dfNodeGens,npGenData = parameters.SolarGenData(dfGenData,int_run_days,int_run_hours,int_minutes, \
                                              intVeHouses,intNumCommNodes,intNumHouseNodes, dctHomeNode)


# now call function to get the SOC constraints array
arrSOCconstraint = parameters.getSOCconstraints(dfNodesTrips,\
                dfTripData, fltCarEff, fltChargeRate, fltDt, int_run_days)

# # call function to get an array of maximum charging rates and map to location by node
arrChargeRate, arrCanCharge,\
arrChargeLoc, arrHomeStation = parameters.MapCarNodesTime(int_minutes,\
                               int_run_hours,int_run_days, intVeHouses,dfNodesTrips,\
                               dfTripData, fltHomeRate, fltWorkRate)


# # call function to get initial SOCs of vehicles
# # replaces first column of soc constraint array
arrSOCconstraint = parameters.getInitialSoc(arrSOCconstraint, fltBatteryCap)  #should this have a different name?

# # call function to get power consumption of the vehicle
arrConsumptionFinal = parameters.findVehicleConsumption(dfNodesTrips, dfTripData, fltCarEff, \
                fltDt, int_run_days)

# #Get Ancillarys service values, the dict has 31 days for dam ru/rd and rtm ru/rd! can do a full run if we want
dctASallprices,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd = parameters.processAsValues(dfAsRDdamPrices,dfAsRUdamPrices,dfAsRDrtmPrices,dfAsRUrtmPrices,int_run_days)

dfNetNodeLoad,arrNetHomeLoad,\
dctNodeIdentity, dctResIdentity = parameters.netLoadChargeLoc(arrHouseGen,arrHouseLoads,\
                                arrChargeLoc,dfNodeLoads,dfNodeGens,intVeHouses,\
                                intTotalNodes,lsNodes,int_minutes,int_run_hours, \
                                int_run_days, arrHomeStation)

#defaults to .1 dispatch, .5 means half the time its Ru and half the time its Rd, over .5 it increases %Ru
dctRuIdentity=dict()
dctRdIdentity=dict()
lsPercentRu=[]
for j in np.linspace(0,1,11):
    j=round(j,1)
    lsPercentRu.append(j)
    lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,j)
    dctRuIdentity[j]=lsRuIdentity
    dctRdIdentity[j]=lsRdIdentity
    
#Default for alll models


#save whatever this parameter runs trip ID info is
dfNodesTrips.to_csv('Output/Tripidentity.csv')




#### Vehicle AS Dispatch

In [46]:
lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,0)

constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost,varSOCs= simulations.make_dispatch_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctNodeIdentity,lsNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,\
                     lsRuIdentity, lsRdIdentity)
constraints, obj_value, varRegUp, varRegDown,varCharge, varDegradationCost =  simulations.make_dispatch_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)
    
# convex usually needs to be minimized
dispatchObj = cvx.Maximize(obj_value)
dispatchProb = cvx.Problem(dispatchObj, constraints)

dispatchProb.solve(solver=cvx.MOSEK)

660.0789811763295

In [ ]:
## charge over time (SOC and charge rate?)
arrRegUp=np.array(varRegUp.value)
arrRegDown=np.array(varRegDown.value)
arrCharging=np.array(varCharge.value)
arrRegDownDispatch=varRegDown.value*lsRdIdentity
arrRegUpDispatch=varRegUp.value*lsRuIdentity
arrDegradation=np.array(varDegradationCost.value)
arrSOCs=np.array(varSOCs.value)
    
        #output array files
np.savetxt(('Output/TOUAsRuns/AS/Charging.csv'),arrCharging, delimiter=",")
np.savetxt(('Output/TOUAsRuns/AS/arrRegUp.csv'),arrRegUp, delimiter=",")
np.savetxt(('Output/TOUAsRuns/AS/arrRegDown.csv'),arrRegDown, delimiter=",")
np.savetxt(('Output/TOUAsRuns/AS/arrRegUpDispatch.csv'),arrRegUpDispatch, delimiter=",")
np.savetxt(('Output/TOUAsRuns/AS/arrRegDownDispatch.csv'),arrRegDownDispatch, delimiter=",")
np.savetxt(('Output/TOUAsRuns/AS/arrSOCs.csv'),arrSOCs, delimiter=",")

#summary values for comparison

lsRegUpVal=[(np.sum(np.sum(arrRegUp,axis=0)*lsDAMruMax))]
lsRegUpDispatchVal=[(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetru)/15)]
lsRegDownVal=[(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdMax))]
lsRegDownDispatchVal=[(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrd)/15)]
lscostElectricityCharge=[(np.sum(lsCostElectric*arrCharging*fltDt))]
lscostElectricityRd=[(np.sum(lsCostElectric*arrRegDown*lsRdIdentity*fltDt/15))]
lsObj=[obj_value.value]
lscostElectricityCharge=[np.sum(arrCharging*lsCostElectric*fltDt)]
lsDegradationCost=[np.sum(arrDegradation)]

dfSummary=pd.DataFrame(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal,\
                                lscostElectricityCharge,lscostElectricityRd, lsDegradationCost),\
                       columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value",\
                               "Cost of Charging", "Cost of RD", "Degradation Cost"])


dfSummary.to_csv("Output/TOUAsRuns/AS/Summary.csv",header=True)  

#produce graphs for each run
fig, ax = plt.subplots(figsize=(11,7))
for ind in range(1,np.shape(arrRegDown)[1]):

    if ind == 1: 
        
        ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegDown[ind,:]).flatten())
    
    else: 
            
        lb = np.sum(arrRegDown[0:ind,:],axis=0)
        ub = np.sum(arrRegDown[0:ind+1,:],axis=0)
       
        ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


plt.ylabel('kW')
plt.xlabel("Interval")
plt.title("Regulation Down")
plt.savefig("Output/TOUAsRuns/AS/regDown.png")
plt.close()
    
fig, ax = plt.subplots(figsize=(11,7))
for ind in range(1,np.shape(arrRegUpDispatch)[1]):

    if ind == 1: 
        
        ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegUp[ind,:]).flatten())
    
    else: 
            
        lb = np.sum(arrRegUp[0:ind,:],axis=0)
        ub = np.sum(arrRegUp[0:ind+1,:],axis=0)
       
        ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


plt.ylabel('kW')
plt.xlabel("Interval")
plt.title("Regulation Up")
plt.savefig("Output/TOUAsRuns/AS/regUp.png")
plt.close()

#### Optimize for TOU only

In [94]:
constraints, varCharge, varSOCs = simulations.make_tou_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,\
                     fltDt, fltBatteryCap, arrConsumptionFinal, PeakLoads,intTotalNodes, dfNetNodeLoad,lsNodes,dctNodeIdentity)

constraints, obj_value, varCharge,varNumberOfCycles, varDegradationCost = simulations.make_tou_objectives(constraints, varSOCs,varCharge,arrSOCconstraint,fltDt,fltBatteryCap,lsCostElectric,arrConsumptionFinal)

touObj = cvx.Minimize(obj_value)
touProb = cvx.Problem(touObj, constraints)

touProb.solve(solver=cvx.MOSEK)

108.71546049312882

In [95]:
#graphs#Calculate flex values
lsFlex = []
arrCharge=np.array(varCharge.value)
for ind, strId in enumerate(dfNodesTrips['TripId']):
    
    intFlexCar = 0
    
    lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()
    for day in range(int_run_days-1):
        lsLocations = lsLocations + lsLocations
 
    intStart = 0
    while intStart < 96*int_run_days and 'A' in lsLocations[intStart:]:

        # Find the first first time we start moving
        intStart = intStart + lsLocations[intStart:].index('A')

        # find how many time steps back we were charging
        intFlex = 0
        strLoc = 'H' # place holder
        while np.round(arrCharge[ind,intStart-intFlex],2) == 0 and strLoc != 'A':
            intFlex += 1
            strLoc = lsLocations[intStart-intFlex]
        intFlexCar += intFlex

        # from there find when we arrive at a destination
        if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
            intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
        elif 'W' in lsLocations[intStart:]:
            intEnd = lsLocations[intStart:].index('W')
        elif 'H' in lsLocations[intStart:]:
            intEnd = lsLocations[intStart:].index('H')
        else: # we are going to assume they are driving the rest of the time??
            break

        intStart += intEnd
        
    lsFlex.append(intFlexCar)


In [96]:
#OUTPUTS
# THINGS TO SAVE !!

#charge over time (SOC and charge rate?)
arrCharging=np.array(varCharge.value)
arrSOC=np.array(varSOCs.value)
#Degradation Cost & Cycles
arrCycles=np.array(varNumberOfCycles.value)
arrDegradation=np.array(varDegradationCost.value) 
#time not charging prior to being away

arrFlex=np.array(lsFlex)


#output array files
np.savetxt(('Output/TOUAsRuns/TOU/arrCharging.csv'),arrCharging, delimiter=",")
np.savetxt(('Output/TOUAsRuns/TOU/arrSOC.csv'),arrSOC, delimiter=",")
np.savetxt(('Output/TOUAsRuns/TOU/Cycles.csv'),arrCycles, delimiter=",")
np.savetxt(('Output/TOUAsRuns/TOU/Degradation.csv'),arrDegradation, delimiter=",")
np.savetxt(('Output/TOUAsRuns/TOU/FlexVals.csv'),arrFlex, delimiter=",")

#summary values for comparison
lsObj=[obj_value.value]
lscostElectricityCharge=[np.sum(arrCharging*lsCostElectric*fltDt)]
lsDegradationCost=[np.sum(arrDegradation)]


dfSummary=pd.DataFrame(zip(lsObj,lscostElectricityCharge,lsDegradationCost),\
                       columns=["Obj Value","Cost of Charging", "Degradation Cost"])

dfSummary.to_csv("Output/TOUAsRuns/TOU/Summary.csv",header=True)  




#### Dispatch Loop over Percent RU

In [8]:
#loop for dispatch quantity

lsObj=[]
lsCycles=[]
lsDegradation=[]
lsRegUpVal=[]
lsRegUpDispatchVal=[]
lsRegDownVal=[]
lsRegDownDispatchVal=[]
lscostElectricityCharge=[]
lscostElectricityRd=[]
lsDegradationCost=[]
lsObjRecalc=[]


dfNumberofCycles=pd.DataFrame()
dfDegradation=pd.DataFrame()


for j in lsPercentRu:
    print(j)
    lsRuIdentity=dctRuIdentity[j]
    lsRdIdentity=dctRdIdentity[j]
    constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost= simulations.make_dispatch_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctNodeIdentity,lsNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,\
                     lsRuIdentity, lsRdIdentity)
    constraints, obj_value, varRegUp, varRegDown,varCharge, varDegradationCost =  simulations.make_dispatch_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)
    
# convex usually needs to be minimized
    dispatchObj = cvx.Maximize(obj_value)
    dispatchProb = cvx.Problem(dispatchObj, constraints)

    dispatchProb.solve(solver=cvx.MOSEK)
    
    ##File outputs
    
    #full arrays, export as csv directly
    arrRegUp=np.array(varRegUp.value)
    arrRegDown=np.array(varRegDown.value)
    arrCharging=np.array(varCharge.value)
    arrSOCs=np.array(varSOCs.value)
    arrRegDownDispatch=varRegDown.value*lsRdIdentity
    arrRegUpDispatch=varRegUp.value*lsRuIdentity
    arrDegradation=np.array(varDegradationCost.value)
    
        #output array files
    np.savetxt(('Output/Dispatch/Charging'+str(j)+'.csv'),arrCharging, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegUp'+str(j)+'.csv'),arrRegUp, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDown'+str(j)+'.csv'),arrRegDown, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    np.savetxt(('Output/Dispatch/arrSOCs'+str(j)+'.csv'),arrSOCs, delimiter=",")
    
    #lists by each vehicle, add as an insert to datatframe
    lsCycles=varNumberOfCycles.value
    lsDegradation=varDegradationCost.value 
    
    #dataframe of list values
    dfNumberofCycles.insert(int(j*10),"Cycles"+str(j),lsCycles,True)
    dfDegradation.insert(int(j*10),"Degradation"+str(j),lsDegradation,True)
   
    #Individual values to save as list, these will zip into dataframe outside of loop
    lsObj.append(obj_value.value)
    RegUpVal=(np.sum(np.sum(arrRegUp,axis=0)*lsDAMruMax))
    RegUpDispatchVal=(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetru)/15)
    RegDownVal=(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdMax))
    RegDownDispatchVal=(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrd)/15)
    costElectricityCharge=(np.sum(lsCostElectric*arrCharging*fltDt))
    costElectricityRd=(np.sum(lsCostElectric*arrRegDown*lsRdIdentity*fltDt/15))
    DegradationCost=(np.sum(arrDegradation))
    ObjRecalc=((RegUpVal+RegUpDispatchVal+RegDownVal+RegDownDispatchVal- \
                       costElectricityCharge-costElectricityRd-DegradationCost))
    lsRegUpVal.append(RegUpVal)
    lsRegUpDispatchVal.append(RegUpDispatchVal)
    lsRegDownVal.append(RegDownVal)
    lsRegDownDispatchVal.append(RegDownDispatchVal)
    lscostElectricityCharge.append(costElectricityCharge)
    lscostElectricityRd.append(costElectricityRd)
    lsDegradationCost.append(DegradationCost)
    lsObjRecalc.append(ObjRecalc)
               
    #produce graphs for each run
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegDownDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegDownDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegDownDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegDownDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Down Dispatch")
    plt.savefig("Output/Dispatch/regDownDispatch"+str(j)+".png")
    plt.close()
    
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegUpDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegUpDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegUpDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegUpDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Up Dispatch")
    plt.savefig("Output/Dispatch/regUpDispatch"+str(j)+".png")
    plt.close()
    
dfSummary=pd.DataFrame(list(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal,\
                                lscostElectricityCharge,lscostElectricityRd, lsDegradationCost,lsObjRecalc)),\
                       columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value",\
                               "Cost of Charging", "Cost of RD", "Degradation Cost", "Objective Recalc"])

#csv export of output values               
dfSummary.to_csv("Output/Dispatch/SummaryValues.csv",header=True)    
dfNumberofCycles.to_csv("Output/Dispatch/Cycles.csv",header=True)    
dfDegradation.to_csv("Output/Dispatch/Degradation.csv",header=True)  

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


#### Optimize a Stationary Battery

In [15]:
lsResNodes = [strNode for strNode in lsNodes if "Res" in strNode]
constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost = simulations.make_battery_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctResIdentity,lsResNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,\
                     lsRuIdentity, lsRdIdentity, fltWorkRate)

constraints,obj_value, varRegUp, varRegDown,varCharge, varDegradationCost = simulations.make_battery_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)


batObj = cvx.Maximize(obj_value)
batProb = cvx.Problem(batObj, constraints)

batProb.solve(solver=cvx.MOSEK)

2659.0105063632545

In [17]:
#loop for dispatch quantity

lsObj=[]
lsCycles=[]
lsDegradation=[]
lsRegUpVal=[]
lsRegUpDispatchVal=[]
lsRegDownVal=[]
lsRegDownDispatchVal=[]
lscostElectricityCharge=[]
lscostElectricityRd=[]
lsDegradationCost=[]
lsObjRecalc=[]


dfNumberofCycles=pd.DataFrame()
dfDegradation=pd.DataFrame()


for j in lsPercentRu:
    print(j)
    lsRuIdentity=dctRuIdentity[j]
    lsRdIdentity=dctRdIdentity[j]
    lsResNodes = [strNode for strNode in lsNodes if "Res" in strNode]
    constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost = simulations.make_battery_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctResIdentity,lsResNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,\
                     lsRuIdentity, lsRdIdentity, fltWorkRate)

    constraints,obj_value, varRegUp, varRegDown,varCharge, varDegradationCost = simulations.make_battery_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)


    batObj = cvx.Maximize(obj_value)
    batProb = cvx.Problem(batObj, constraints)

    batProb.solve(solver=cvx.MOSEK)
    
    ##File outputs
    
    #full arrays, export as csv directly
    arrRegUp=np.array(varRegUp.value)
    arrRegDown=np.array(varRegDown.value)
    arrCharging=np.array(varCharge.value)
    arrRegDownDispatch=varRegDown.value*lsRdIdentity
    arrRegUpDispatch=varRegUp.value*lsRuIdentity
    arrDegradation=np.array(varDegradationCost.value)
    arrSOCs=np.array(varSOCs,value)
    
        #output array files
    np.savetxt(('Output/Battery/Charging'+str(j)+'.csv'),arrCharging, delimiter=",")
    np.savetxt(('Output/Battery/arrRegUp'+str(j)+'.csv'),arrRegUp, delimiter=",")
    np.savetxt(('Output/Battery/arrRegDown'+str(j)+'.csv'),arrRegDown, delimiter=",")
    np.savetxt(('Output/Battery/arrRegUpDispatch'+str(j)+'.csv'),arrRegUpDispatch, delimiter=",")
    np.savetxt(('Output/Battery/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    np.savetxt(('Output/Battery/arrSOCs'+str(j)+'.csv'),arrSOCs, delimiter=",")
    
    #lists by each vehicle, add as an insert to datatframe
    lsCycles=varNumberOfCycles.value
    lsDegradation=varDegradationCost.value 
    
    #dataframe of list values
    dfNumberofCycles.insert(int(j*10),"Cycles"+str(j),lsCycles,True)
    dfDegradation.insert(int(j*10),"Degradation"+str(j),lsDegradation,True)
   
    #Individual values to save as list, these will zip into dataframe outside of loop
    lsObj.append(obj_value.value)
    RegUpVal=(np.sum(np.sum(arrRegUp,axis=0)*lsDAMruMax))
    RegUpDispatchVal=(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetru)/15)
    RegDownVal=(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdMax))
    RegDownDispatchVal=(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrd)/15)
    costElectricityCharge=(np.sum(lsCostElectric*arrCharging*fltDt))
    costElectricityRd=(np.sum(lsCostElectric*arrRegDown*lsRdIdentity*fltDt/15))
    DegradationCost=(np.sum(arrDegradation))
    ObjRecalc=((RegUpVal+RegUpDispatchVal+RegDownVal+RegDownDispatchVal- \
                       costElectricityCharge-costElectricityRd-DegradationCost))
    lsRegUpVal.append(RegUpVal)
    lsRegUpDispatchVal.append(RegUpDispatchVal)
    lsRegDownVal.append(RegDownVal)
    lsRegDownDispatchVal.append(RegDownDispatchVal)
    lscostElectricityCharge.append(costElectricityCharge)
    lscostElectricityRd.append(costElectricityRd)
    lsDegradationCost.append(DegradationCost)
    lsObjRecalc.append(ObjRecalc)
               
    #produce graphs for each run
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegDownDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegDownDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegDownDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegDownDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Down Dispatch")
    plt.savefig("Output/Battery/regDownDispatch"+str(j)+".png")
    plt.close()
    
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegUpDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegUpDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegUpDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegUpDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Up Dispatch")
    plt.savefig("Output/Battery/regUpDispatch"+str(j)+".png")
    plt.close()
    
dfSummary=pd.DataFrame(list(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal,\
                                lscostElectricityCharge,lscostElectricityRd, lsDegradationCost,lsObjRecalc)),\
                       columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value",\
                               "Cost of Charging", "Cost of RD", "Degradation Cost", "Objective Recalc"])
#csv export of output values               
dfSummary.to_csv("Output/Battery/SummaryValues.csv",header=True)    
dfNumberofCycles.to_csv("Output/Battery/Cycles.csv",header=True)    
dfDegradation.to_csv("Output/Battery/Degradation.csv",header=True)  

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


#### 30 Daysssss

In [52]:
dctASallprices,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd = parameters.processAsValues(dfAsRDdamPrices,dfAsRUdamPrices,dfAsRDrtmPrices,dfAsRUrtmPrices,1)

dfDAMrd=dctASallprices["DAMrd"]
lsDAMrd=np.array(dfDAMrd.values.reshape(96*30,1))

dfDAMru=dctASallprices["DAMru"]
lsDAMru=np.array(dfDAMru.values.reshape(96*30,1))

dfRTMrd=dctASallprices["RTMrd"]
lsRTMrd=np.array(dfRTMrd.values.reshape(96*30,1))

dfRTMru=dctASallprices["RTMru"]
lsRTMru=np.array(dfRTMru.values.reshape(96*30,1))

lsNetru=lsRTMru-lsDAMru
lsNetru[lsNetru<0]=0
lsNetrd=lsRTMrd-lsDAMrd
lsNetrd[lsNetrd<0]=0


In [53]:
varInit=np.ones(np.shape(arrSOCconstraint)[0])*100


arrRegupStack=np.empty(np.shape(arrSOCconstraint))
arrRegDownStack=np.empty(np.shape(arrSOCconstraint))
arrChargingStack=np.empty(np.shape(arrSOCconstraint))
arrSOCsStack=np.empty(np.shape(arrSOCconstraint))
arrRegDownDispatchStack=np.empty(np.shape(arrSOCconstraint))
arrRegUpDispatchStack=np.empty(np.shape(arrSOCconstraint))
arrDegradationStack=np.empty(np.shape(arrSOCconstraint)[0])
TotalCycles=np.empty(np.shape(arrSOCconstraint)[0])


lsObj=[]
lsCycles=[]
lsDegradation=[]
lsRegUpVal=[]
lsRegUpDispatchVal=[]
lsRegDownVal=[]
lsRegDownDispatchVal=[]
lscostElectricityCharge=[]
lscostElectricityRd=[]
lsDegradationCost=[]
lsObjRecalc=[]
#iterate over day sets!!!!
for n in range(30):
    print(n)
    i=n*1*96
    j=i+1*96
#it was doing that weird list of list values thing again...
    lsDAMrdrun=list()
    lsDAMrurun=list()
    lsNetrurun=list()
    lsNetrdrun=list()
    lsDAMrdholder=lsDAMrd[i:j].tolist()
    lsDAMruholder=lsDAMru[i:j].tolist()
    lsNetruholder=lsNetru[i:j].tolist()
    lsNetrdholder=lsNetrd[i:j].tolist()
    
    for i in lsDAMrdholder:
        lsDAMrdrun.append(i[0])
    for i in lsDAMruholder:
        lsDAMrurun.append(i[0])
    for i in lsNetruholder:
        lsNetrurun.append(i[0])
    for i in lsNetrdholder:    
        lsNetrdrun.append(i[0])
    
    lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,0.2)

    constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost, varSOCs= simulations.month_run_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctNodeIdentity,lsNodes, \
                         arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                         fltBatteryCap,fltDt,lsCostElectric,\
                         lsRuIdentity, lsRdIdentity,varInit)
    constraints, obj_value, varRegUp, varRegDown,varCharge, varDegradationCost =  simulations.month_run_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                        lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdrun,lsDAMrurun,lsNetrurun,lsNetrdrun,fltDt)

    # convex usually needs to be minimized
    MonthRunObj = cvx.Maximize(obj_value)
    MonthRunProb = cvx.Problem(MonthRunObj, constraints)

    MonthRunProb.solve(solver=cvx.MOSEK)
    
    print(obj_value.value)

    arrRegup=np.array(varRegUp.value)
    arrRegDown=np.array(varRegDown.value)
    arrCharging=np.array(varCharge.value)
    arrSOCs=np.array(varSOCs.value)
    varInit=arrSOCs[:,-1]
    arrRegDownDispatch=varRegDown.value*lsRdIdentity
    arrRegUpDispatch=varRegUp.value*lsRuIdentity
    arrDegradation=np.array(varDegradationCost.value)
    Cycles=np.array(varNumberOfCycles.value)
    
    arrRegupStack=np.hstack((arrRegupStack,arrRegup))
    arrRegDownStack=np.hstack((arrRegDownStack,arrRegDown))
    arrChargingStack=np.hstack((arrChargingStack,arrCharging))
    arrSOCsStack=np.hstack((arrSOCsStack,arrSOCs))
    arrRegDownDispatchStack=np.hstack((arrRegDownDispatchStack,arrRegDownDispatch))
    arrRegUpDispatchStack=np.hstack((arrRegUpDispatchStack,arrRegUpDispatch))
    arrDegradationStack=np.hstack((arrDegradationStack,arrDegradation))

            #output array files

    
    TotalCycles=Cycles+TotalCycles
    
        #Individual values to save as list, these will zip into dataframe outside of loop
    lsObj.append(obj_value.value)
    RegUpVal=(np.sum(np.sum(arrRegup,axis=0)*lsDAMrurun))
    RegUpDispatchVal=(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetrurun)/15)
    RegDownVal=(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdrun))
    RegDownDispatchVal=(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrdrun)/15)
    costElectricityCharge=(np.sum(lsCostElectric*arrCharging*fltDt))
    costElectricityRd=(np.sum(lsCostElectric*arrRegDown*lsRdIdentity*fltDt/15))
    DegradationCost=(np.sum(arrDegradation))
    ObjRecalc=((RegUpVal+RegUpDispatchVal+RegDownVal+RegDownDispatchVal- \
                           costElectricityCharge-costElectricityRd-DegradationCost))
    lsRegUpVal.append(RegUpVal)
    lsRegUpDispatchVal.append(RegUpDispatchVal)
    lsRegDownVal.append(RegDownVal)
    lsRegDownDispatchVal.append(RegDownDispatchVal)
    lscostElectricityCharge.append(costElectricityCharge)
    lscostElectricityRd.append(costElectricityRd)
    lsDegradationCost.append(DegradationCost)
    lsObjRecalc.append(ObjRecalc)
    
lsObj.append(np.sum(np.array(lsObj)))
lsRegUpVal.append(np.sum(np.array(lsRegUpVal)))
lsRegUpDispatchVal.append(np.sum(np.array(lsRegUpDispatchVal)))
lsRegDownVal.append(np.sum(np.array(lsRegDownVal)))
lsRegDownDispatchVal.append(np.sum(np.array(lsRegDownDispatchVal)))
lscostElectricityCharge.append(np.sum(np.array(lscostElectricityCharge)))
lscostElectricityRd.append(np.sum(np.array(lscostElectricityRd)))
lsDegradationCost.append(np.sum(np.array(lsDegradationCost)))
lsObjRecalc.append(np.sum(np.array(lsObjRecalc)))
                               
dfSummary=pd.DataFrame(list(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal,\
                                    lscostElectricityCharge,lscostElectricityRd, lsDegradationCost,lsObjRecalc)),\
                           columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value",\
                                   "Cost of Charging", "Cost of RD", "Degradation Cost", "Objective Recalc"])
    #csv export of output values               
dfSummary.to_csv("Output/MonthRun/SummaryValues.csv",header=True)    
np.savetxt(('Output/MonthRun/Charging.csv'),arrChargingStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrRegUp.csv'),arrRegupStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrRegDown.csv'),arrRegDownStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrRegUpDispatch.csv'),arrRegUpDispatchStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrRegDownDispatch.csv'),arrRegDownDispatchStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrRegDownDispatch.csv'),arrRegDownDispatchStack, delimiter=",")
np.savetxt(('Output/MonthRun/arrSOCs.csv'),arrSOCsStack, delimiter=",")
np.savetxt(('Output/MonthRun/Cycles.csv'),TotalCycles, delimiter=",")
np.savetxt(('Output/MonthRun/Degradation.csv'),arrDegradationStack, delimiter=",")

0
16.807290401560497
1
25.144620844961455
2
28.855692203754142
3
8.537917595820195
4
25.43405078950493
5
37.016156518025575
6
18.87748859620412
7
31.688389896095828
8
17.15576106310131
9
41.775395664196104
10
11.904181671037588
11
14.574923073138258
12
36.02810309969152
13
57.30698639321609
14
31.97171998097653
15
42.48415861629799
16
42.68794447613865
17
50.82790976321743
18
65.75154557731753
19
113.0531608075997
20
83.87760854745999
21
122.04653907904427
22
181.83178618715056
23
307.1539187706762
24
303.4081606291715
25
122.31220085174078
26
95.88763285424298
27
97.98688993251386
28
53.69749955251664
29
72.78309187729359


In [34]:
#NO LONGER RELEVANT
int_minutes=int(15)
int_run_days=30 # day
int_run_hours=24 # hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  # convert kw data to kwh

dfLoadData = pd.read_csv("Data/use_data_31days.csv") #31, 1 min increment, 142 houses 

lsCostElectric = []
lsHolder = dfCostElectric.values.tolist()
for ind in range(int_run_days):
    for n in lsHolder:
        lsCostElectric.append(n[0])

In [36]:
#NO LONGER RELEVANT FOR MULTIPLE RUN of 2 DAY
arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, \
lsNodes,dctHomeNode = parameters.convertLoadData(dfLoadData,dfNodesTrips, \
                                    int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes)

#create generation for 15 min increments for each node
arrHouseGen,dfNodeGens,npGenData = parameters.SolarGenData(dfGenData,int_run_days,int_run_hours,int_minutes, \
                                              intVeHouses,intNumCommNodes,intNumHouseNodes, dctHomeNode)

dctASallprices,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd = parameters.processAsValues(dfAsRDdamPrices,dfAsRUdamPrices,dfAsRDrtmPrices,dfAsRUrtmPrices,int_run_days)


# now call function to get the SOC constraints array
arrSOCconstraint = parameters.getSOCconstraints(dfNodesTrips,\
                dfTripData, fltCarEff, fltChargeRate, fltDt, int_run_days)

# # call function to get an array of maximum charging rates and map to location by node
arrChargeRate, arrCanCharge,\
arrChargeLoc, arrHomeStation = parameters.MapCarNodesTime(int_minutes,\
                               int_run_hours,int_run_days, intVeHouses,dfNodesTrips,\
                               dfTripData, fltHomeRate, fltWorkRate)


# # call function to get initial SOCs of vehicles
# # replaces first column of soc constraint array
arrSOCconstraint = parameters.getInitialSoc(arrSOCconstraint, fltBatteryCap)  #should this have a different name?

# # call function to get power consumption of the vehicle
arrConsumptionFinal = parameters.findVehicleConsumption(dfNodesTrips, dfTripData, fltCarEff, \
                fltDt, int_run_days)

# #Get Ancillarys service values, the dict has 31 days for dam ru/rd and rtm ru/rd! can do a full run if we want

dfNetNodeLoad,arrNetHomeLoad,\
dctNodeIdentity, dctResIdentity = parameters.netLoadChargeLoc(arrHouseGen,arrHouseLoads,\
                                arrChargeLoc,dfNodeLoads,dfNodeGens,intVeHouses,\
                                intTotalNodes,lsNodes,int_minutes,int_run_hours, \
                                int_run_days, arrHomeStation)

#defaults to .1 dispatch, .5 means half the time its Ru and half the time its Rd, over .5 it increases %Ru
lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,.2)

In [ ]:
constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost, varSOCs= simulations.make_dispatch_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctNodeIdentity,lsNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,\
                     lsRuIdentity, lsRdIdentity)
constraints, obj_value, varRegUp, varRegDown,varCharge, varDegradationCost =  simulations.make_dispatch_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrd,lsDAMru,lsNetru,lsNetrd,fltDt)
    
# convex usually needs to be minimized
MonthRunObj = cvx.Maximize(obj_value)
MonthRunProb = cvx.Problem(MonthRunObj, constraints)

MonthRunProb.solve(solver=cvx.MOSEK)

In [31]:
print(varSOCs.value)




None


In [ ]:
#TOU and dispatch comparison runs. Compare SOC and flex time values for each




